# Import Modules


In [1]:
import pandas as pd
import numpy as np
import sys
import os
import warnings
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, normalize
from sklearn import metrics
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append(os.path.abspath(os.path.join('../scripts')))

from clean_telecom_dataframe import Telecom
from plot_telecom_dataframe import TelecomPlotter

plot = TelecomPlotter()
cleaner = Telecom()


# Read CSV Data


In [2]:
telecom_clean_df = pd.read_csv("../data/telecom_cleaned_data.csv")

In [4]:
# Relevant columns for exprience analysis
telecom_exprience_df = telecom_clean_df[['MSISDN/Number','Avg RTT DL (ms)','Avg RTT UL (ms)','Avg Bearer TP DL (kbps)','Avg Bearer TP UL (kbps)','TCP DL Retrans. Vol (Bytes)','TCP UL Retrans. Vol (Bytes)','Handset Type']]
telecom_exprience_df

,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type
0,33664962239,42.000000,5.000000,23.0,44.0,2.080991e+07,759658.664811,Samsung Galaxy A5 Sm-A520F
1,33681854413,65.000000,5.000000,16.0,26.0,2.080991e+07,759658.664811,Samsung Galaxy J5 (Sm-J530)
2,33760627129,109.795706,17.662883,6.0,9.0,2.080991e+07,759658.664811,Samsung Galaxy A8 (2018)
3,33699795932,109.795706,17.662883,6.0,9.0,2.080991e+07,759658.664811,Samsung Sm-G390F
4,33665368271,102.000000,5.000000,22.0,38.0,2.080991e+07,759658.664811,Huawei Mate 20 Lite
...,...,...,...,...,...,...,...,...
138066,33645655643,109.795706,17.662883,55.0,54.0,2.080991e+07,759658.664811,Samsung Galaxy J5 (Sm-J530)
138067,33650688697,32.000000,0.000000,52.0,65.0,2.080991e+07,759658.664811,Apple iPhone 8 Plus (A1897)
138068,33663449963,27.000000,2.000000,23.0,54.0,2.080991e+07,759658.664811,Apple iPhone Se (A1723)
138069,33621890103,43.000000,6.000000,43.0,47.0,2.080991e+07,759658.664811,Apple iPhone Xs (A2097)


In [5]:
# merge Avg RTT (ms) , Avg Bearer TP (kbps) and Total TCP Retrans. Vol (Bytes)
telecom_exprience_df['Total Avg RTT (ms)'] = telecom_exprience_df['Avg RTT DL (ms)'] + telecom_exprience_df['Avg RTT UL (ms)']
telecom_exprience_df['Total Avg Bearer TP (kbps)'] = telecom_exprience_df['Avg Bearer TP DL (kbps)'] + telecom_exprience_df['Avg Bearer TP UL (kbps)']
telecom_exprience_df['Total TCP Retrans. Vol (Bytes)'] = telecom_exprience_df['TCP DL Retrans. Vol (Bytes)'] + telecom_exprience_df['TCP UL Retrans. Vol (Bytes)']
telecom_exprience_df.head()

/tmp/ipykernel_32657/1313543884.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  telecom_exprience_df['Total Avg RTT (ms)'] = telecom_exprience_df['Avg RTT DL (ms)'] + telecom_exprience_df['Avg RTT UL (ms)']
/tmp/ipykernel_32657/1313543884.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  telecom_exprience_df['Total Avg Bearer TP (kbps)'] = telecom_exprience_df['Avg Bearer TP DL (kbps)'] + telecom_exprience_df['Avg Bearer TP UL (kbps)']
/tmp/ipykernel_32657/1313543884.py:4: SettingWithCopyWarning: 
A v

,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
0,33664962239,42.000000,5.000000,23.0,44.0,2.080991e+07,759658.664811,Samsung Galaxy A5 Sm-A520F,47.000000,67.0,2.156957e+07
1,33681854413,65.000000,5.000000,16.0,26.0,2.080991e+07,759658.664811,Samsung Galaxy J5 (Sm-J530),70.000000,42.0,2.156957e+07
2,33760627129,109.795706,17.662883,6.0,9.0,2.080991e+07,759658.664811,Samsung Galaxy A8 (2018),127.458589,15.0,2.156957e+07
3,33699795932,109.795706,17.662883,6.0,9.0,2.080991e+07,759658.664811,Samsung Sm-G390F,127.458589,15.0,2.156957e+07
4,33665368271,102.000000,5.000000,22.0,38.0,2.080991e+07,759658.664811,Huawei Mate 20 Lite,107.000000,60.0,2.156957e+07


In [6]:
telecom_exprience_df = telecom_exprience_df[['MSISDN/Number','Total Avg RTT (ms)','Total Avg Bearer TP (kbps)','Total TCP Retrans. Vol (Bytes)','Handset Type']]
telecom_exprience_df.head()

,MSISDN/Number,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
0,33664962239,47.000000,67.0,2.156957e+07,Samsung Galaxy A5 Sm-A520F
1,33681854413,70.000000,42.0,2.156957e+07,Samsung Galaxy J5 (Sm-J530)
2,33760627129,127.458589,15.0,2.156957e+07,Samsung Galaxy A8 (2018)
3,33699795932,127.458589,15.0,2.156957e+07,Samsung Sm-G390F
4,33665368271,107.000000,60.0,2.156957e+07,Huawei Mate 20 Lite


In [7]:
tellco_exprience_df1 = telecom_exprience_df.groupby(
    'MSISDN/Number').agg({'Total Avg RTT (ms)': 'sum', 'Total Avg Bearer TP (kbps)': 'sum', 'Total TCP Retrans. Vol (Bytes)': 'sum','Handset Type': [lambda x: x.mode()[0]] }) #' '.join(x)
tellco_exprience_df1

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
,sum,sum,sum,<lambda>
MSISDN/Number,,,,
33601001722,46.000000,76.0,2.156957e+07,Huawei P20 Lite Huawei Nova 3E
33601001754,31.000000,99.0,2.156957e+07,Apple iPhone 7 (A1778)
33601007832,84.000000,248.0,7.607247e+05,Apple iPhone 5S (A1457)
33601008617,119.000000,56844.0,3.094040e+07,Apple iPhone Se (A1723)
33601010682,151.458589,7908.0,2.233199e+07,Samsung Galaxy A8 (2018)
...,...,...,...,...
33789960306,127.458589,106.0,2.156957e+07,Samsung Galaxy S8 Plus (Sm-G955F)
33789967113,53.000000,67.0,2.156957e+07,Apple iPhone 6S (A1688)
